**Web Apps**

# Reading Checkpoint

In [ ]:
!pip install tensorflow_hub
!pip install keras tf-models-official pydot graphviz

In [ ]:
import os

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

from keras.utils import np_utils

import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization as tokenization

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt


In [ ]:


from google.colab import drive

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {`s`
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow import keras

In [ ]:
path = "/content/gdrive/My Drive/checkpoint" 
optimizer = nlp.optimization.create_optimizer(
    2e-5, num_train_steps=417
, num_warmup_steps=41)
model = keras.models.load_model(path, custom_objects={'AdamWeightDecay': optimizer})

INFO:absl:using Adamw optimizer
INFO:absl:gradient_clip_norm=1.000000


INFO:absl:gradient_clip_norm=1.000000


In [ ]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2",
                            trainable=True)

In [ ]:

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)


In [ ]:
def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(string_list, tokenizer, max_seq_length):
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [ ]:
tokenizer

In [ ]:
pip install transformers==3


# Voice emotion extraction

In [ ]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split
!python --version
#!pip install pipwin
#!pipwin install pyaudio
!apt-get install libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0
!apt-get install ffmpeg libav-tools
!pip install pyaudio

Python 3.7.10
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libasound2-dev' instead of 'libasound-dev'
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (195 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 160772 files and directories currently installed.

In [ ]:
import pyaudio
import os
import wave
import pickle
from sys import byteorder
from array import array
from struct import pack
from sklearn.neural_network import MLPClassifier 
!pip install SpeechRecognition
import speech_recognition as sr

    # load the saved model (after training)
def get_audio_sentiments(file_wv):
    loading_model=open("/content/gdrive/My Drive/voice_model/mlp_classifier.model", "rb")
    model = pickle.load(loading_model)
    file_wav=open("/content/gdrive/My Drive/roboart/03-01-02-01-02-02-01.wav","rb")

    features = extract_feature(file_wav, mfcc=True, chroma=True, mel=True).reshape(1, -1)
    result = model.predict(features)[0]
   
   


    print("sentiment:", result)
    return result





     |████████████████████████████████| 32.8MB 140kB/s 


In [ ]:
# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "happy",
}


In [ ]:

def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result


# Text

In [ ]:
from textblob import TextBlob
def question_polarity(question):
  blob = TextBlob(question)
  if blob.sentiment.polarity<0:

    return ".But It Appear you are not feeling good wanna talk with a real human?"
  elif blob.sentiment.polarity>0:
    return "😁"
  else:
    return " "

#Face Emotion detection


In [ ]:
from keras.models import load_model
from IPython.display import display, Javascript
from google.colab.output import eval_js
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np


In [ ]:
face_classifier = cv2.CascadeClassifier('/content/gdrive/My Drive/haarcascade_frontalface_default.xml')
classifier = load_model('/content/gdrive/My Drive/Emotion_Detection.h5')

class_labels = ['Angry','Happy','Neutral','Sad','Surprise']

In [ ]:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
emotions=[]
emotions

[]

In [ ]:
emotions

[]

In [ ]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

In [ ]:
##################################### mohema tala3ha foo2
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [ ]:
##################################################################################      Working         ##############################################################################
from collections import Counter
label_html = 'Capturing...'

# initialze bounding box to empty
bbox = ''
count = 0 
def read_face_emotion():
  video_stream()
  emotions=[]
  
  while True:

      js_reply = video_frame(label_html, bbox)
      labels=[]
      if not js_reply:
          break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])

      # create transparent overlay for bounding box
      bbox_array = np.zeros([480,640,4], dtype=np.uint8)

      # grayscale image for face detection
      gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

      # get face region coordinates
      faces = face_cascade.detectMultiScale(gray)
      # get face bounding box for overlay
      for (x,y,w,h) in faces:
        bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        

        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)
            preds = classifier.predict(roi)[0]
           # print("\nprediction = ",preds)
            label=class_labels[preds.argmax()]
          #  print("\nprediction max = ",preds.argmax())
          #  print("\nlabel = ",label)

            emotions.append(label)
            try:
              if(len(emotions)%7==0):
                
                return most_frequent(emotions)

              
            except Exception as e:
              continue
               


            

            label_position = (x,y)
            cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)

  # return(mode(emotions))

    
      # convert overlay of bbox into bytes
      
      # update bbox so next frame gets new overlay
    

In [ ]:
#print('most common '+read_face_emotion())

In [ ]:
# def most_common(List):
#     return(mode(List))
#read_face_emotion()
# def threaded_face_emotion():
#   read_face_emotion()
#   most_common(emotions)
  ################################### faksana

In [ ]:
# import threading

# t1=threading.Thread(target=read_face_emotion)
# t1.start()
# print(threading.active_count() )
# print('fierst')

# print('hello')

In [ ]:
# video_stream()
  ################################### faksana  ################################### faksana  ################################### faksana  ################################### faksana
# # label for video
# label_html = 'Front Camera'
# # initialze bounding box to empty
# bbox = ''
# count = 0 

# emotions=[]
# ###################################
# def face_emotion():
#   video_stream()
#   print('out')
#   while True:
#     print('in')
#     js_reply = video_frame(label_html, bbox)
#     print('2222222222')
#     if not js_reply:
#         print('not')
#         return
#     labels = []
#     print('1')
#     # convert JS response to OpenCV Image
#     img = js_to_image(js_reply["img"])
#     print('2')
#     # create transparent overlay for bounding box
#     bbox_array = np.zeros([480,640,4], dtype=np.uint8)
#     print('3')
#     # grayscale image for face detection
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     print('4')
#     # get face region coordinates
#     faces = face_cascade.detectMultiScale(gray)
#     #faces = face_classifier.detectMultiScale(gray,1.3,5)


#     print('before loop')

#     # get face bounding box for overlay
#     for (x,y,w,h) in faces:
#         print('in for loop')
#        # cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
#         roi_gray = gray[y:y+h,x:x+w]
#         roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        
#         if np.sum([roi_gray])!=0:
#             roi = roi_gray.astype('float')/255.0
#             roi = img_to_array(roi)
#             roi = np.expand_dims(roi,axis=0)

#         # make a prediction on the ROI, then lookup the class

#             preds = classifier.predict(roi)[0]
#             print("\nprediction = ",preds)
#             label=class_labels[preds.argmax()]
#             emotions.append(label)
#             fs=True
#             print(fs)
#             print(len(emotions))
#             try:
#               print('in try')
#               print(len(emotions))
#               if len(emotions)%5==0:
#                 print('most common emotion: '+most_common(emotions))
#                 fs=False
#                 emotions.clear()
#             except Exception as e:
#               print('unexpected error in face detection')
#               print("\nprediction max = ",preds.argmax())
#             #print("\nlabel = ",label)
#             label_position = (x,y)
#             cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
#         else:
#             cv2.putText(img,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
#             print("else")
#     # update bbox so next frame gets new overlay
# face_emotion()
  ################################### faksana  ################################### faksana  ################################### faksana  ################################### faksana

# Q&A model now


In [ ]:
from transformers import BertTokenizer
from transformers import BertForQuestionAnswering
import torch
import textwrap
from textblob import TextBlob
import nltk

Load the Pretrained Model for Question Answering Bert

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer1 = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

model1 = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

#AutoTokenizer.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")

In [ ]:
wrapper = textwrap.TextWrapper(width=80)
import concurrent.futures


def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
 

    


    input_ids = tokenizer1.encode(question, answer_text)


    sep_index = input_ids.index(tokenizer1.sep_token_id)


    num_seg_a = sep_index + 1

    num_seg_b = len(input_ids) - num_seg_a

    segment_ids = [0] * num_seg_a + [1] * num_seg_b

    assert len(segment_ids) == len(input_ids)

    start_scores, end_scores = model1(torch.tensor([input_ids]), 
                                     token_type_ids=torch.tensor(
                                         [segment_ids]))  


    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)


    tokens = tokenizer1.convert_ids_to_tokens(input_ids)


    answer = tokens[answer_start]


    for i in range(answer_start + 1, answer_end + 1):


        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]


        else:
            answer += ' ' + tokens[i]
    if "CLS" in answer:
      print('You: '+ question )
      print('Sorry i dont know')
      return 'Sorry i cant handle that'
    elif "[SEP]" in answer:
      print('You: '+ question )
      print('Sorry i dont know')
      return 'Sorry i dont understand that yet'
    else:
      
      print('You: '+ question )
      print('Answer : '+ answer )
      return answer



In [ ]:
def answer_this(question,y=1000):
  
  if(y<200):
    return
 # question = ['which types of vaccines available to cure covid']
  global Selected_Category
  global inputs
  global model
  global prediction
  question1=[question]
  inputs = bert_encode(string_list=list(question1), 
                     tokenizer=tokenizer, 
                     max_seq_length=63)
  prediction = model.predict(inputs)
  possible_categories=['Comparison','Economic_Effects','Having_COVID','Individual_Response','Nomenclature','Origin','Prevention','Reporting','Societal Effects','Societal Response','Speculation','Symptoms','Testing','Transmission','Treatment']
  for i in range (len(possible_categories)):
    if np.argmax(prediction)== i:

      Selected_Category=possible_categories[i]



  # if np.argmax(prediction)== 0:
  #   Selected_Category='Comparison'
    
  # if np.argmax(prediction)== 1:
  #   Selected_Category='Economic_Effects'
    
  # if np.argmax(prediction)== 2:
  #   Selected_Category='Having_COVID'
    
  # if np.argmax(prediction)== 3:
  #   Selected_Category='Individual_Response'
    
  # if np.argmax(prediction)== 4:
  #   Selected_Category='Nomenclature'
    
  # if np.argmax(prediction)== 5:
  #   Selected_Category='Origin'
    
  # if np.argmax(prediction)== 6:
  #   Selected_Category='Prevention'
    
  # if np.argmax(prediction)== 7:
  #   Selected_Category='Reporting'
    
  # if np.argmax(prediction)== 8:
  #   Selected_Category='Societal Effects'
    
  # if np.argmax(prediction)== 9:
  #   Selected_Category='Societal Response'
    
  # if np.argmax(prediction)== 10:
  #   Selected_Category='Speculation'
    
  # if np.argmax(prediction)== 11:
  #   Selected_Category='Symptoms'
    
  # if np.argmax(prediction)== 12:
  #   Selected_Category='Testing'
    
  # if np.argmax(prediction)== 13:
  #   Selected_Category='Transmission'
    
  # if np.argmax(prediction)== 14:
  #   Selected_Category='Treatment'
    

  
  flag_exception = False

  try:
    with open('/content/gdrive/My Drive/Categories/'+Selected_Category+'.txt') as f:
      x=f.readlines()
    Categorial_Abtract=x
    bert_abstract=''.join(Categorial_Abtract)

    bert_abstract
  except Exception as e:
    print('You: '+ question )
    print(Selected_Category +' Category is still not available yet.')
    return "Category is still not available yet"
    flag_exception = True

  if  flag_exception == True:
    print(Selected_Category +' Category is still not available yet.')
    return "Category is still not available yet"
  else:
    try:
      
      
      #print(bert_abstract)
      return answer_question(question,bert_abstract[:y])
    except Exception as e:
      print ('ops an error occured  '+ str(e))
      answer_this(question,y-100)
      

  

In [ ]:
# answer_this('what is covid cure')
# answer_this('and where did covid start')
# answer_this('what is the symptoms?')
# answer_this('what is the incubation period?')
# answer_this('when will corona end?')
answer_this('what is sars')

You: what is sars
Answer : severe acute respiratory syndrome coronavirus 2


'severe acute respiratory syndrome coronavirus 2'

In [ ]:
import concurrent.futures
import time
  #print(read_face_emotion())
message=input()  
#message='what is the treatment'

with concurrent.futures.ThreadPoolExecutor() as executor:
      future = executor.submit(read_face_emotion)
      future2=executor.submit(answer_this,message)
        
      r=future.result()
      print('your face is '+r)
    
      # time.sleep(5)
      # if r is not True:

      #   future.cancel()
     # concurrent.futures.ThreadPoolExecutor.shutdown()

  
      

In [ ]:
read_face_emotion()

In [ ]:

  #answer_this('what is covid cure')
# answer_this('and where did covid start')
# answer_this('what is the symptoms?')
# answer_this('what is the incubation period?')
# answer_this('how does corona spread?')



# Web App


In [ ]:


from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
h='hello'

In [ ]:
from flask import Flask
!pip install flask-ngrok
!pip install flask-wtf
! pip install wtforms==2.2.1
!pip install pyngrok
!ngrok authtoken 1tqnKSRNTcXG8w4GrKSnjYvPVlv_3G54329xNgebzaRBUpGeq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:

from flask import Flask,redirect,url_for,render_template,request,jsonify
from flask_wtf import FlaskForm
from flask_ngrok import run_with_ngrok
#!ngrok authtoken 1tqnKSRNTcXG8w4GrKSnjYvPVlv_3G54329xNgebzaRBUpGeq

from wtforms import StringField, PasswordField, SubmitField, BooleanField
from wtforms.validators import DataRequired, Length, Email, EqualTo, ValidationError
app = Flask(__name__,template_folder='/content/gdrive/MyDrive/roboart',static_folder='/content/gdrive/MyDrive/roboart/static',)
app._static_folder = '/content/gdrive/MyDrive/roboart/static'
run_with_ngrok(app)   
app.config['SECRET_KEY'] = '5791628bb0b13ce0c676dfde585ba245'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///site.db'

@app.route("/pricing")
def pricing():
    return render_template('pricing.html')    
@app.route("/team")
def team():
    return render_template('team.html')  


@app.route("/faq")
def faq():
    return render_template('faq.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')
@app.route("/")
@app.route("/index")
def home():
    return render_template('index-2.html')
@app.route("/about")
def about():
    return render_template('about.html')

@app.route('/answer/voice')
def answer_voice():
 
  audio_wav=request.args.get("audio")
  # r = sr.Recognizer()
  # with sr.AudioFile(audio_wav) as source:

  #   audio_data = r.record(source)

  # text = r.recognize_google(audio_data)
  # print(text)
  result=get_audio_sentiments(audio_wav)
  return jsonify({'emotion':result  })




@app.route("/chatbotmic")
def chatbotmic():
    return render_template('chatbotmic.html')    
@app.route("/chatbotvideo")
def chatbotvideo():
    return render_template('chatbotvid.html')    


@app.route("/login", methods=['GET', 'POST'])
def login():
    
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(email=form.email.data).first()
        if user and bcrypt.check_password_hash(user.password, form.password.data):
            login_user(user, remember=form.remember.data)
            next_page = request.args.get('next')
            return redirect(next_page) if next_page else redirect(url_for('home'))
        else:
            flash('Login Unsuccessful. Please check email and password', 'danger')
    return render_template('login.html', title='Login', form=form)




class LoginForm(FlaskForm):
    email = StringField('Email',
                        validators=[DataRequired(), Email()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember = BooleanField('Remember Me')
    submit = SubmitField('Login')


@app.route("/chatbot-text",methods=["GET"])

def chatbot():
    return render_template('chatbottext.html')







@app.route("/chatbot/answer", methods=['GET'])
def get_bot_response():
  
  
    
  question = request.args.get("question",0,type=str)
  print("testing 3 message = "+question )
  answer=answer_this(question)
  print("testing" +answer)
  sent= question_polarity(question)
  print(sent)
  final=answer+" "+sent  
  return jsonify({'response':final})

  
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c8343b9c031a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:26] "GET /static/css/responsive.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:26] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:26] "GET /static/css/owl.transitions.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:26] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:26] "GET /static/css/animate.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:26] "GET /static/css/owl.carousel.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:27] "GET /static/css/meanmenu.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:27] "GET /static/css/font-awesome.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:52:27] "GET /static/css/icon.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13

testing 3 message = what is covid?
You: what is covid?
Societal Response Category is still not available yet.
testingCategory is still not available yet
 
testing 3 message = what is covid


...

You: where did covid start? :D
Answer : wuhan , china
testingwuhan , china
😁
testing 3 message = i feel sick


INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:54:32] "GET /chatbot/answer?question=i%20feel%20sick HTTP/1.1" 200 -


You: i feel sick
Sorry i dont know
testingSorry i dont understand that yet
.But It Appear you are not feeling good wanna talk with a real human?


INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:54:42] "GET /pricing HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:54:43] "GET /static/fonts/fontawesome-webfont.woff2?v=4.5.0 HTTP/1.1" 500 -
ERROR:werkzeug:Error on request:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 323, in run_wsgi
    execute(self.server.app)
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 314, in execute
    for data in application_iter:
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/wsgi.py", line 579, in __next__
    data = self.file.read(self.buffer_size)
PermissionError: [Errno 1] Operation not permitted
INFO:werkzeug:127.0.0.1 - - [13/Jun/2021 08:54:44] "GET /static/fonts/fontawesome-webfont.woff?v=4.5.0 HTTP/1.1" 500 -
ERROR:werkzeug:Error on request:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 323, in run_wsgi
    execute(self.server.ap

sentiment: happy


In [ ]:
!ngrok authtoken 1tqnKSRNTcXG8w4GrKSnjYvPVlv_3G54329xNgebzaRBUpGeq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
answer

NameError: ignored



# End of Code

In [ ]:
# import concurrent.futures

# def foo():
#   return 2

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     future = executor.submit(foo)
#     return_value = future.result()
#     print(return_value)